In [1]:
import os
import pickle
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from utils_data import  Vocabulary, Vectorizer, HeadQA, HeadQA_IR
from utils_data import parse_dataset, parse_ir_dataset, random_oversamplig, random_undersampling
from utils_data import filter_by_category, save_dataset_to_pickle, load_dataset_from_pickle

from training import get_optimizer, train, train_ir, validate, validate_ir, evaluator, evaluator_ir, evaluate
from training import load_embeddings_from_file, make_embedding_matrix
from training import pad_seq, encoder_bert, encoder_bert_ir, encoder_bert_instance, encoder_bert_ir_instance
from training import evaluator_bert, evaluator_bert_ir, evaluate_better

from supervised_models import LogisticRegression, BasicLSTM, BiLSTM_model
from ir_models import LSTM_QA, LSTM_CNN_QA, BERT_QA

%matplotlib inline
%load_ext autoreload
%autoreload 2

C:\Users\tec005m\Anaconda3\envs\afi\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
CATEGORY = 'pharmacology'

In [3]:
from datasets import load_dataset

data_es = load_dataset('head_qa', 'es' )
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

Reusing dataset head_qa (C:\Users\tec005m\.cache\huggingface\datasets\head_qa\es\1.1.0\473dc5357942a3ff52963bd73cad0d167bd1bbc1ca5ca0732ee7372b480dd735)


### Modelos supervisados puros

In [36]:
training_instances = load_dataset_from_pickle('../data/training.pickle')
validation_instances = load_dataset_from_pickle('../data/validation.pickle')
testing_instances = load_dataset_from_pickle('../data/testing.pickle')

mixed_training = load_dataset_from_pickle('../data/mixed_oversampling_training.pickle')

In [37]:
training_categ = filter_by_category(mixed_training, category=CATEGORY)
validation_categ = filter_by_category(validation_instances, category=CATEGORY)
testing_categ = filter_by_category(testing_instances, category=CATEGORY)

dev_categ = filter_by_category(validation, category=CATEGORY)
test_categ = filter_by_category(testing, category=CATEGORY)

In [38]:
vectorizer = Vectorizer.vectorize_training(training_categ)
vocab = vectorizer.sentence_vocab
label_vocab = vectorizer.label_vocab

In [39]:
trainset = HeadQA(instances=training_categ, vectorizer=vectorizer, right_padding=False, max_length=30)
validset = HeadQA(instances=validation_categ, vectorizer=vectorizer, right_padding=False, max_length=30)
testset = HeadQA(instances=testing_categ, vectorizer=vectorizer, right_padding=False, max_length=30)

In [40]:
batch_size = 32
train_dt = DataLoader(trainset, batch_size=batch_size,drop_last=True)
valid_dt = DataLoader(validset, batch_size=batch_size,drop_last=True)
test_dt = DataLoader(testset, batch_size=batch_size,drop_last=True)

#### Logistic Regressor

In [9]:
torch.random.manual_seed(42)
logistic_regressor = LogisticRegression(trainset.max_length, 1)
optimizer = get_optimizer(logistic_regressor, lr = 0.01, wd = 1e-5)

In [10]:
training_results = train(logistic_regressor, optimizer, train_dt, valid_dt, validate, epochs=30)

C:\Users\tec005m\Anaconda3\envs\afi\lib\site-packages\torch\nn\functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 0 train loss  43.4278 valid loss 1.225 and accuracy 0.7221
Epoch 1 train loss  42.8614 valid loss 0.921 and accuracy 0.7411
Epoch 2 train loss  42.1228 valid loss 0.921 and accuracy 0.7411
Epoch 3 train loss  42.1228 valid loss 0.921 and accuracy 0.7411
Epoch 4 train loss  42.1228 valid loss 0.921 and accuracy 0.7411
Epoch 5 train loss  42.1228 valid loss 0.921 and accuracy 0.7411
Epoch 6 train loss  42.1228 valid loss 0.921 and accuracy 0.7411
Epoch 7 train loss  42.1228 valid loss 0.921 and accuracy 0.7411
Epoch 8 train loss  42.1228 valid loss 0.921 and accuracy 0.7411
Epoch 9 train loss  42.1228 valid loss 0.921 and accuracy 0.7411
Epoch 10 train loss  42.1228 valid loss 0.921 and accuracy 0.7411
Epoch 11 train loss  42.1228 valid loss 0.921 and accuracy 0.7411
Epoch 12 train loss  42.1228 valid loss 0.921 and accuracy 0.7411
Epoch 13 train loss  42.1228 valid loss 0.921 and accuracy 0.7411
Epoch 14 train loss  42.1228 valid loss 0.921 and accuracy 0.7411
Epoch 15 train loss 

In [11]:
acc, points = evaluate(logistic_regressor, dev_categ, trainset.encode, evaluator)
print(f'DEV Dominio: {CATEGORY}')
print(f'accuracy: {acc}, points: {points}')
print('----------')
acc, points = evaluate(logistic_regressor, test_categ, trainset.encode, evaluator)
print(f'TEST Dominio: {CATEGORY}')
print(f'accuracy: {acc}, points: {points}')

DEV Dominio: pharmacology
accuracy: tensor([0.2133]), points: -33
----------
TEST Dominio: pharmacology
accuracy: tensor([0.2407]), points: -17


In [12]:
model_path = f'trained_models_v2/logistic_regressor_{CATEGORY}'
torch.save(logistic_regressor.state_dict(), model_path)

#### LSTM

In [13]:
lstm = BasicLSTM(len(vocab), 64, trainset.max_length, 1, embedding_dim=100)
optimizer = get_optimizer(lstm, lr = 0.001, wd = 1e-5)

In [14]:
training_results = train(lstm, optimizer, train_dt, valid_dt, validate, epochs=50)

Epoch 0 train loss  0.6613 valid loss 0.027 and accuracy 0.2500
Epoch 1 train loss  0.7151 valid loss 0.025 and accuracy 0.2500
Epoch 2 train loss  0.6968 valid loss 0.025 and accuracy 0.2500
Epoch 3 train loss  0.6915 valid loss 0.025 and accuracy 0.2500
Epoch 4 train loss  0.6882 valid loss 0.024 and accuracy 0.2511
Epoch 5 train loss  0.6854 valid loss 0.024 and accuracy 0.2567
Epoch 6 train loss  0.6825 valid loss 0.024 and accuracy 0.2667
Epoch 7 train loss  0.6775 valid loss 0.024 and accuracy 0.2723
Epoch 8 train loss  0.6718 valid loss 0.024 and accuracy 0.2768
Epoch 9 train loss  0.6616 valid loss 0.024 and accuracy 0.2891
Epoch 10 train loss  0.6360 valid loss 0.023 and accuracy 0.3270
Epoch 11 train loss  0.6033 valid loss 0.025 and accuracy 0.3571
Epoch 12 train loss  0.5353 valid loss 0.025 and accuracy 0.3973
Epoch 13 train loss  0.4856 valid loss 0.024 and accuracy 0.5357
Epoch 14 train loss  0.4248 valid loss 0.025 and accuracy 0.5536
Epoch 15 train loss  0.3780 valid l

In [15]:
acc, points = evaluate(lstm, dev_categ, trainset.encode, evaluator)
print(f'DEV Dominio: {CATEGORY}')
print(f'accuracy: {acc}, points: {points}')
print('----------')
acc, points = evaluate(lstm, test_categ, trainset.encode, evaluator)
print(f'TEST Dominio: {CATEGORY}')
print(f'accuracy: {acc}, points: {points}')

DEV Dominio: pharmacology
accuracy: tensor([0.2178]), points: -29
----------
TEST Dominio: pharmacology
accuracy: tensor([0.2757]), points: 47


In [16]:
model_path = f'trained_models_v2/lstm_{CATEGORY}'
torch.save(lstm.state_dict(), model_path)

#### BiLSTM

In [41]:
word_to_idx = load_dataset_from_pickle('trained_models/biomedical_embeddings/word_to_index.pickle')
embeddings = load_dataset_from_pickle('trained_models/biomedical_embeddings/wordvectors.pickle')
embedding_file = "trained_models/biomedical_embeddings/Scielo_wiki_FastText300.vec"
words = vocab.vocab2index.keys()
embedding_matrix = make_embedding_matrix(embedding_file, list(words), word_to_idx, embeddings)

In [18]:
bilstm = BiLSTM_model(embedding_matrix.shape[1], embedding_matrix.shape[0], 1, 
                     pretrained_embeddings=embedding_matrix, max_length=trainset.max_length)
optimizer = get_optimizer(bilstm, lr = 0.01, wd = 1e-5)

C:\Users\tec005m\Anaconda3\envs\afi\lib\site-packages\torch\nn\modules\rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [19]:
training_results = train(bilstm, optimizer, train_dt, valid_dt, validate, epochs=50)

Epoch 0 train loss  0.4418 valid loss 2.762 and accuracy 0.2500
Epoch 1 train loss  57.7020 valid loss 2.762 and accuracy 0.2500
Epoch 2 train loss  57.7020 valid loss 2.762 and accuracy 0.2500
Epoch 3 train loss  57.7020 valid loss 2.762 and accuracy 0.2500
Epoch 4 train loss  57.7020 valid loss 2.762 and accuracy 0.2500
Epoch 5 train loss  57.7020 valid loss 2.762 and accuracy 0.2500
Epoch 6 train loss  57.7020 valid loss 2.762 and accuracy 0.2500
Epoch 7 train loss  57.7020 valid loss 2.762 and accuracy 0.2500
Epoch 8 train loss  57.7020 valid loss 2.762 and accuracy 0.2500
Epoch 9 train loss  57.7020 valid loss 2.762 and accuracy 0.2500
Epoch 10 train loss  57.7020 valid loss 2.762 and accuracy 0.2500
Epoch 11 train loss  57.7020 valid loss 2.762 and accuracy 0.2500
Epoch 12 train loss  57.7020 valid loss 2.762 and accuracy 0.2500
Epoch 13 train loss  57.7020 valid loss 2.762 and accuracy 0.2500
Epoch 14 train loss  57.7020 valid loss 2.762 and accuracy 0.2500
Epoch 15 train loss  

In [20]:
acc, points = evaluate(bilstm, dev_categ, trainset.encode, evaluator)
print(f'DEV Dominio: {CATEGORY}')
print(f'accuracy: {acc}, points: {points}')
print('----------')
acc, points = evaluate(bilstm, test_categ, trainset.encode, evaluator)
print(f'TEST Dominio: {CATEGORY}')
print(f'accuracy: {acc}, points: {points}')

DEV Dominio: pharmacology
accuracy: tensor([0.2133]), points: -33
----------
TEST Dominio: pharmacology
accuracy: tensor([0.2363]), points: -25


In [21]:
model_path = f'trained_models_v2/bilstm_{CATEGORY}'
torch.save(bilstm.state_dict(), model_path)

### Modelos supervisados IR

In [45]:
training_instances = load_dataset_from_pickle('../data/training_ir.pickle')
validation_instances = load_dataset_from_pickle('../data/validation_ir.pickle')
testing_instances = load_dataset_from_pickle('../data/testing_ir.pickle')
mixed_training_ir = load_dataset_from_pickle('../data/mixed_oversampling_training_ir.pickle')

In [46]:
training_categ = filter_by_category(mixed_training_ir, category=CATEGORY)
validation_categ = filter_by_category(validation_instances, category=CATEGORY)
testing_categ = filter_by_category(testing_instances, category=CATEGORY)

dev_categ = filter_by_category(validation, category=CATEGORY)
test_categ = filter_by_category(testing, category=CATEGORY)

In [47]:
vectorizer = Vectorizer.vectorize_ir_dataset(training_categ)
vocab = vectorizer.sentence_vocab
label_vocab = vectorizer.label_vocab

In [48]:
trainset = HeadQA_IR(instances=training_categ, vectorizer=vectorizer, right_padding=False, max_length=15)
validset = HeadQA_IR(instances=validation_categ, vectorizer=vectorizer, right_padding=False, max_length=15)
testset = HeadQA_IR(instances=testing_categ, vectorizer=vectorizer, right_padding=False, max_length=15)

In [49]:
batch_size = 32
train_dt = DataLoader(trainset, batch_size=batch_size,drop_last=True)
valid_dt = DataLoader(validset, batch_size=batch_size,drop_last=True)
test_dt = DataLoader(testset, batch_size=batch_size,drop_last=True)

In [50]:
word_to_idx = load_dataset_from_pickle('trained_models/biomedical_embeddings/word_to_index_ir.pickle')
embeddings = load_dataset_from_pickle('trained_models/biomedical_embeddings/wordvectors_ir.pickle')
embedding_file = "trained_models/biomedical_embeddings/Scielo_wiki_FastText300.vec"
words = vocab.vocab2index.keys()
embedding_matrix = make_embedding_matrix(embedding_file, list(words), word_to_idx, embeddings)

#### LSTM-QA

In [28]:
lstm_qa = LSTM_QA(vocab_size=len(vocab), hidden_size=64, x_size=trainset.max_length, n_classes=1, embedding_size=300,
               pretrained_embeddings=embedding_matrix)
optimizer = get_optimizer(lstm_qa, lr = 0.001, wd = 1e-5)

Loading pretrained embeddings...


In [29]:
training_results = train_ir(lstm_qa, optimizer, train_dt, valid_dt, validate_ir, epochs=50)

Epoch 0 train loss  0.4408 valid loss 0.180 and accuracy 0.2500
Epoch 1 train loss  0.8875 valid loss 0.038 and accuracy 0.2500
Epoch 2 train loss  0.6999 valid loss 0.038 and accuracy 0.2500
Epoch 3 train loss  0.7168 valid loss 0.028 and accuracy 0.2500
Epoch 4 train loss  0.6747 valid loss 0.030 and accuracy 0.2500
Epoch 5 train loss  0.6721 valid loss 0.030 and accuracy 0.2500
Epoch 6 train loss  0.6713 valid loss 0.029 and accuracy 0.2500
Epoch 7 train loss  0.6642 valid loss 0.030 and accuracy 0.2500
Epoch 8 train loss  0.6708 valid loss 0.029 and accuracy 0.2500
Epoch 9 train loss  0.6674 valid loss 0.028 and accuracy 0.2500
Epoch 10 train loss  0.6809 valid loss 0.028 and accuracy 0.2500
Epoch 11 train loss  0.6686 valid loss 0.028 and accuracy 0.2500
Epoch 12 train loss  0.6643 valid loss 0.028 and accuracy 0.2500
Epoch 13 train loss  0.6634 valid loss 0.029 and accuracy 0.2500
Epoch 14 train loss  0.6615 valid loss 0.031 and accuracy 0.2500
Epoch 15 train loss  0.6656 valid l

In [30]:
acc, points = evaluate(lstm_qa, dev_categ, trainset.encode, evaluator_ir)
print(f'DEV Dominio: {CATEGORY}')
print(f'accuracy: {acc}, points: {points}')
print('----------')
acc, points = evaluate(lstm_qa, test_categ, trainset.encode, evaluator_ir)
print(f'TEST Dominio: {CATEGORY}')
print(f'accuracy: {acc}, points: {points}')

DEV Dominio: pharmacology
accuracy: tensor([0.2133]), points: -33
----------
TEST Dominio: pharmacology
accuracy: tensor([0.2385]), points: -21


In [31]:
model_path = f'trained_models_v2/lstm_qa_{CATEGORY}'
torch.save(lstm_qa.state_dict(), model_path)

#### LSTM-QA/CNN

In [32]:
lstm_cnn_qa = LSTM_CNN_QA(vocab_size=len(vocab), hidden_size=64, x_size=trainset.max_length, n_classes=1, embedding_size=300,
               pretrained_embeddings=embedding_matrix)
optimizer = get_optimizer(lstm_cnn_qa, lr = 0.001, wd = 1e-5)

Loading pretrained embeddings...


In [33]:
training_results = train_ir(lstm_cnn_qa, optimizer, train_dt, valid_dt, validate_ir, epochs=50)

Epoch 0 train loss  0.4362 valid loss 0.205 and accuracy 0.2500
Epoch 1 train loss  0.8291 valid loss 0.075 and accuracy 0.2500
Epoch 2 train loss  0.8159 valid loss 0.031 and accuracy 0.2522
Epoch 3 train loss  0.7409 valid loss 0.026 and accuracy 0.2500
Epoch 4 train loss  0.6913 valid loss 0.028 and accuracy 0.2500
Epoch 5 train loss  0.6899 valid loss 0.025 and accuracy 0.2500
Epoch 6 train loss  0.6913 valid loss 0.025 and accuracy 0.2500
Epoch 7 train loss  0.6901 valid loss 0.025 and accuracy 0.2500
Epoch 8 train loss  0.6886 valid loss 0.025 and accuracy 0.2500
Epoch 9 train loss  0.6918 valid loss 0.025 and accuracy 0.2500
Epoch 10 train loss  0.6933 valid loss 0.026 and accuracy 0.2500
Epoch 11 train loss  0.6953 valid loss 0.026 and accuracy 0.2500
Epoch 12 train loss  0.6988 valid loss 0.026 and accuracy 0.2500
Epoch 13 train loss  0.7046 valid loss 0.027 and accuracy 0.2500
Epoch 14 train loss  0.7136 valid loss 0.026 and accuracy 0.2500
Epoch 15 train loss  0.7021 valid l

In [34]:
acc, points = evaluate(lstm_cnn_qa, dev_categ, trainset.encode, evaluator_ir)
print(f'DEV Dominio: {CATEGORY}')
print(f'accuracy: {acc}, points: {points}')
print('----------')
acc, points = evaluate(lstm_cnn_qa, test_categ, trainset.encode, evaluator_ir)
print(f'TEST Dominio: {CATEGORY}')
print(f'accuracy: {acc}, points: {points}')

DEV Dominio: pharmacology
accuracy: tensor([0.2222]), points: -25
----------
TEST Dominio: pharmacology
accuracy: tensor([0.2560]), points: 11


In [35]:
model_path = f'trained_models_v2/lstm_cnn_qa_{CATEGORY}'
torch.save(lstm_cnn_qa.state_dict(), model_path)

### Evaluacion

In [44]:
logistic_regressor = LogisticRegression(trainset.max_length, 1)
lstm = BasicLSTM(len(vocab), 64, trainset.max_length, 1, embedding_dim=100)
bilstm = BiLSTM_model(embedding_matrix.shape[1], embedding_matrix.shape[0], 1, 
                     pretrained_embeddings=embedding_matrix, max_length=trainset.max_length)

models = [logistic_regressor, lstm, bilstm]
paths = [f'trained_models_v2/logistic_regressor_{CATEGORY}', 
         f'trained_models_v2/lstm_{CATEGORY}',         
         f'trained_models_v2/bilstm_{CATEGORY}']

print(paths[0])

for i, model in enumerate(models):
    model.load_state_dict(torch.load(paths[i]))
    model.eval()
    acc, points, acc_list, points_list = evaluate_better(model, dev_categ, trainset.encode, evaluator)
    print('DEV')
    print('Accuracy media', acc)
    print('Puntos media', points)
    print(acc_list)
    print(points_list)
    print('---------')
    acc, points, acc_list, points_list = evaluate_better(model, test_categ, trainset.encode, evaluator)
    print('TEST')
    print('Accuracy media', acc)
    print('Puntos media', points)
    print(acc_list)
    print(points_list)
    print('---------')
    print() 

trained_models_v2/logistic_regressor_pharmacology
DEV
Accuracy media 0.21333334
Puntos media -33.0
[tensor(0.2133)]
[-33]
---------
TEST
Accuracy media 0.24068221
Puntos media -8.5
[tensor(0.2325), tensor(0.2489)]
[-16, -1]
---------

DEV
Accuracy media 0.21777777
Puntos media -29.0
[tensor(0.2178)]
[-29]
---------
TEST
Accuracy media 0.27564543
Puntos media 23.5
[tensor(0.2456), tensor(0.3057)]
[-4, 51]
---------

DEV
Accuracy media 0.21333334
Puntos media -33.0
[tensor(0.2133)]
[-33]
---------
TEST
Accuracy media 0.23630583
Puntos media -12.5
[tensor(0.2281), tensor(0.2445)]
[-20, -5]
---------



In [51]:
lstm_qa = LSTM_QA(vocab_size=len(vocab), hidden_size=64, x_size=trainset.max_length, n_classes=1, embedding_size=300,
               pretrained_embeddings=embedding_matrix)
lstm_cnn_qa = LSTM_CNN_QA(vocab_size=len(vocab), hidden_size=64, x_size=trainset.max_length, n_classes=1, embedding_size=300,
               pretrained_embeddings=embedding_matrix)

models = [lstm_qa, lstm_cnn_qa]

paths = [f'trained_models_v2/lstm_qa_{CATEGORY}',
         f'trained_models_v2/lstm_cnn_qa_{CATEGORY}'
        ]

for i, model in enumerate(models):
    model.load_state_dict(torch.load(paths[i]))
    model.eval()
    acc, points, acc_list, points_list = evaluate_better(model, dev_categ, trainset.encode, evaluator_ir)
    print('DEV')
    print('Accuracy media', acc)
    print('Puntos media', points)
    print(acc_list)
    print(points_list)
    print('---------')
    acc, points, acc_list, points_list = evaluate_better(model, test_categ, trainset.encode, evaluator_ir)
    print('TEST')
    print('Accuracy media', acc)
    print('Puntos media', points)
    print(acc_list)
    print(points_list)
    print('---------')
    print()  

Loading pretrained embeddings...
Loading pretrained embeddings...
DEV
Accuracy media 0.21333334
Puntos media -33.0
[tensor(0.2133)]
[-33]
---------
TEST
Accuracy media 0.23851797
Puntos media -10.5
[tensor(0.2412), tensor(0.2358)]
[-8, -13]
---------

DEV
Accuracy media 0.22222222
Puntos media -25.0
[tensor(0.2222)]
[-25]
---------
TEST
Accuracy media 0.25604266
Puntos media 5.5
[tensor(0.2675), tensor(0.2445)]
[16, -5]
---------

